In [20]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

## Persiapan Dataset

In [5]:
file_path = "./dataset/klasifikasimhs.csv"
df = pd.read_csv(file_path)

In [14]:
df.head()
df.info()
missing_value = df.isnull().sum()
print(missing_value)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Tempat Tinggal               100 non-null    int64 
 1   Pekerjaan Orang Tua          100 non-null    object
 2   Penghasilan Orang Tua        100 non-null    int64 
 3   Jumlah Tanggungan Orang Tua  100 non-null    int64 
 4   Kendaraan                    100 non-null    int64 
 5   Kelayakan Keringanan UKT     100 non-null    object
dtypes: int64(4), object(2)
memory usage: 4.8+ KB
Tempat Tinggal                 0
Pekerjaan Orang Tua            0
Penghasilan Orang Tua          0
Jumlah Tanggungan Orang Tua    0
Kendaraan                      0
Kelayakan Keringanan UKT       0
dtype: int64


## Normalisasi Dataset Dengan LabelEncoder

In [23]:
le = LabelEncoder()

df_object = ["Pekerjaan Orang Tua", "Kelayakan Keringanan UKT"]

for kolom in df_object:
  df[kolom] = le.fit_transform(df[kolom])

print(df.head())
df.info()

   Tempat Tinggal  Pekerjaan Orang Tua  Penghasilan Orang Tua  \
0               0                    4               10000000   
1               0                    6                8000000   
2               1                    5                4000000   
3               1                    3                3000000   
4               0                    0                2000000   

   Jumlah Tanggungan Orang Tua  Kendaraan  Kelayakan Keringanan UKT  
0                            3          1                         0  
1                            2          2                         1  
2                            4          0                         0  
3                            5          1                         0  
4                            2          1                         1  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                   

## Logika Fuzzy Mamdani

In [26]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# 1. Definisikan variabel fuzzy input
penghasilan = ctrl.Antecedent(np.arange(0, 11000000, 100000), 'penghasilan')
tanggungan = ctrl.Antecedent(np.arange(0, 8, 1), 'tanggungan')
kendaraan = ctrl.Antecedent(np.arange(0, 5, 1), 'kendaraan')

# 2. Definisikan variabel fuzzy output
kelayakan = ctrl.Consequent(np.arange(0, 101, 1), 'kelayakan')

# 3. Definisikan membership functions input sesuai rentang
penghasilan['rendah'] = fuzz.trimf(penghasilan.universe, [0, 0, 3000000])
penghasilan['sedang'] = fuzz.trimf(penghasilan.universe, [2000000, 4000000, 6000000])
penghasilan['tinggi'] = fuzz.trimf(penghasilan.universe, [5000000, 10000000, 10000000])

tanggungan['sedikit'] = fuzz.trimf(tanggungan.universe, [0, 0, 2])
tanggungan['sedang'] = fuzz.trimf(tanggungan.universe, [1, 3, 4])
tanggungan['banyak'] = fuzz.trimf(tanggungan.universe, [3, 7, 7])

kendaraan['tidak_ada'] = fuzz.trimf(kendaraan.universe, [0, 0, 0])
kendaraan['sedikit'] = fuzz.trimf(kendaraan.universe, [0, 1, 1])
kendaraan['banyak'] = fuzz.trimf(kendaraan.universe, [1, 5, 5])

# 4. Membership function output (kelayakan)
kelayakan['tidak_layak'] = fuzz.trimf(kelayakan.universe, [0, 0, 50])
kelayakan['layak'] = fuzz.trimf(kelayakan.universe, [50, 100, 100])

# 5. Definisikan aturan fuzzy (rules)
rule1 = ctrl.Rule(penghasilan['rendah'] & tanggungan['banyak'] & kendaraan['tidak_ada'], kelayakan['layak'])
rule2 = ctrl.Rule(penghasilan['rendah'] & tanggungan['banyak'] & kendaraan['sedikit'], kelayakan['layak'])
rule3 = ctrl.Rule(penghasilan['rendah'] & tanggungan['sedang'], kelayakan['layak'])
rule4 = ctrl.Rule(penghasilan['sedang'] & tanggungan['banyak'], kelayakan['layak'])
rule5 = ctrl.Rule(penghasilan['tinggi'], kelayakan['tidak_layak'])
rule6 = ctrl.Rule(penghasilan['sedang'] & tanggungan['sedikit'], kelayakan['tidak_layak'])
rule7 = ctrl.Rule(kendaraan['banyak'], kelayakan['tidak_layak'])

# 6. Buat sistem kontrol fuzzy
kelayakan_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7])
kelayakan_sim = ctrl.ControlSystemSimulation(kelayakan_ctrl)


## Simulasi Kelayakan Mahasiswa

In [25]:
# 7. Contoh penggunaan simulasi
kelayakan_sim.input['penghasilan'] = 2500000
kelayakan_sim.input['tanggungan'] = 4
kelayakan_sim.input['kendaraan'] = 1

kelayakan_sim.compute()

print(f"Nilai kelayakan fuzzy: {kelayakan_sim.output['kelayakan']:.2f}")

# 8. Tentukan keputusan berdasarkan nilai output fuzzy
if kelayakan_sim.output['kelayakan'] >= 50:
    print("Status: Layak Keringanan UKT")
else:
    print("Status: Tidak Layak Keringanan UKT")

Nilai kelayakan fuzzy: 77.98
Status: Layak Keringanan UKT
